### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
import pandas as pd
from save_pkl import *

In [2]:
import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)

<module 'resonator_quality_factor_fit' from 'D:\\qtlab_replacement\\scripts\\resonator_quality_factor_fit.py'>

In [3]:
pna = Agilent_N5242A('pna', address = 'PNA')

# Прицеливание

In [13]:
sample_name = 'M-M MPLX 1'
min_freq = 2.5e9
max_freq = 12e9

In [14]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(min_freq, max_freq)
pna.set_nop(32001)
pna.set_bandwidth(1000)
pna.set_power(-10)

True

In [15]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.diff(phase)/(freqs[1]-freqs[0])
delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

save_pkl({'type':'S21 overview near', 'name': sample_name}, (freqs, ([0]), np.log10(S21[0])*10, S21[1]))

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


AttributeError: 'tuple' object has no attribute 'items'

In [20]:
threshold = .1

In [22]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = np.abs(np.diff(data_complex))
data_diff[data_diff<threshold] = 0
plt.plot(freqs[1:], data_diff)
peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[1:][peaks]
print (peak_freqs)

found 59 peaks
[  2.87732812e+09   3.21754688e+09   3.58121875e+09   3.58300000e+09
   3.58389062e+09   3.58507812e+09   3.60170312e+09   3.60229688e+09
   3.60318750e+09   3.96804688e+09   4.50123438e+09   4.50153125e+09
   5.05520312e+09   5.05550000e+09   5.55900000e+09   5.78700000e+09
   6.30860938e+09   6.30890625e+09   7.12768750e+09   7.51184375e+09
   7.51303125e+09   7.51421875e+09   7.67156250e+09   7.89689062e+09
   7.89718750e+09   8.36506250e+09   8.36535938e+09   8.90270312e+09
   8.90300000e+09   9.59531250e+09   1.02745625e+10   1.02888125e+10
   1.02891094e+10   1.05812344e+10   1.05815312e+10   1.05821250e+10
   1.09659844e+10   1.09662812e+10   1.13317344e+10   1.13462812e+10
   1.13471719e+10   1.13480625e+10   1.13489531e+10   1.13516250e+10
   1.13525156e+10   1.13531094e+10   1.13537031e+10   1.13554844e+10
   1.15184688e+10   1.15187656e+10   1.15190625e+10   1.15196562e+10
   1.15540938e+10   1.16820469e+10   1.17351875e+10   1.17366719e+10
   1.19807031e+10  

# Куда сохранять

In [6]:
%matplotlib qt4
data_root = get_location_pkl()
if not os.path.exists('{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)):
    os.mkdir('{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name))

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

# Мерить зонаторы

In [7]:
resonator_freqs = [6.07471875e+09, 6.38578125e+09, 6.47868750e+09, 6.76668750e+09,\
   6.88921875e+09, 7.15621875e+09, 7.29187500e+09, 7.54040625e+09, 7.69781250e+09,\
   7.92318750e+09, 8.10225000e+09, 8.29696875e+09]
powers = np.linspace(0, -80, 81) # длинный скан
#powers = np.linspace(0, -30, 31) # короткий скан

In [8]:
resonator_freq_span = 2e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(201)
    pna.set_average(1)
    pna.set_averages(4)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='notch_port')
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

Started at:  Apr 17 2017 20:31:59
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.72 s
d:\qtlab\ReiData\data/2017-04-17/20-32-00/
Started at:  Apr 17 2017 21:44:46
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.12 s
d:\qtlab\ReiData\data/2017-04-17/21-44-46/
Started at:  Apr 17 2017 22:57:31
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 43.13 s
d:\qtlab\ReiData\data/2017-04-17/22-57-31/
Started at:  Apr 18 2017 00:10:18
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 53.78 s
d:\qtlab\ReiData\data/2017-04-18/00-10-18/
Started at:  Apr 18 2017 01:23:15
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 43.2 s
d:\qtlab\ReiData\data/2017-04-18/01-23-16/
Started at:  Apr 18 2017 02:36:02
Time left: 

c:\python36\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 200.
  warnings.warn(errors[info][0], RuntimeWarning)


Started at:  Apr 18 2017 05:01:35
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.41 s
d:\qtlab\ReiData\data/2017-04-18/05-01-35/
Started at:  Apr 18 2017 06:14:21
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.5 s
d:\qtlab\ReiData\data/2017-04-18/06-14-21/
Started at:  Apr 18 2017 07:27:09
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.59 s
d:\qtlab\ReiData\data/2017-04-18/07-27-09/
Started at:  Apr 18 2017 08:39:55
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.69 s
d:\qtlab\ReiData\data/2017-04-18/08-39-55/
Started at:  Apr 18 2017 09:52:41
Time left: 0 h 0 m 0.0 s	, parameter values: Power:    -80	, average cycle time:   54s	s	
Elapsed time: 1 h 12 m 42.5 s
d:\qtlab\ReiData\data/2017-04-18/09-52-41/


# Загрузить фиты и построить графики

In [9]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 6
columns = 2
figs, axes = plt.subplots(rows,columns, figsize=(15, 20))
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
resonator_type='notch_port'
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        print (fitdata, resonator_id)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<1000000]
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        
        Q_ax = axes[(resonator_id)%rows, (resonator_id)//rows]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_ax.annotate('$f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}, $Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])), \
                          (0.1, 0.1), xycoords='axes fraction', bbox={'alpha':1.0, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e4, 1e6])
        if (resonator_id==0):
            plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=2, framealpha=1)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()

    Unnamed: 0    Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  180310.080259  67514.864447      2489.061169  71837.085786   
1            1  179465.744919  68259.082404      1775.132179  72638.021910   
2            2  178724.352505  68820.732774      1587.954223  73298.757363   
3            3  178873.615914  69591.523517      1448.987263  74201.004003   
4            4  178459.655750  70048.078692      1124.880812  74725.388637   
5            5  178106.339607  70560.049466       976.105929  75300.742882   
6            6  178198.810141  71238.116271      1014.099695  76047.540537   
7            7  177680.136629  71628.453665      1041.290960  76503.743039   
8            8  177528.926311  72012.294246      1016.494082  76955.008048   
9            9  177514.647288  72361.049346      1207.259302  77326.895851   
10          10  177169.456984  72704.223112      1126.063396  77727.680290   
11          11  180303.850414  74279.268490       548.945021  79

    Unnamed: 0    Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  142683.053877   99063.244909       511.192575  120992.080432   
1            1  142225.466642   99921.891637       356.493326  122164.959815   
2            2  141977.116101  100548.238901       331.943956  123153.718654   
3            3  142221.674807  101333.725384       220.983321  124416.724187   
4            4  141950.303374  101954.171784       362.077522  125295.248662   
5            5  142251.545895  102568.231269       145.499639  126387.617250   
6            6  142005.313212  103019.992297       133.978039  126947.324254   
7            7  142371.575217  103350.218580       159.108928  127580.007211   
8            8  142034.735230  103859.411029       244.854981  128276.314447   
9            9  141842.862186  104177.857281       209.949552  128743.374124   
10          10  142305.691019  104527.545762       266.779987  129292.725466   
11          11  141616.450337  104194.66

[81 rows x 17 columns] 6
    Unnamed: 0    Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  166686.655124  55910.926631       290.733869  60953.024369   
1            1  165807.649075  56314.105910       239.042057  61433.412912   
2            2  165906.966369  56831.609048       247.793104  62076.003915   
3            3  165696.933770  57207.433285       215.744141  62506.970338   
4            4  165210.919054  57442.949011       235.480558  62779.637364   
5            5  165116.811193  57667.319060       292.608459  63033.943515   
6            6  164640.323715  57801.548697       397.204262  63172.761125   
7            7  165078.251533  57975.539947       440.744252  63425.875141   
8            8  165504.040017  58500.163309       592.898810  64024.809489   
9            9  164825.773761  58620.704567       527.413943  64163.519344   
10          10  165219.655709  59204.562366       756.121282  64905.446737   
11          11  165639.792630  59755.67

    Unnamed: 0   Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  23972.203553  14838.544160       714.020026  14838.576470   
1            1  23190.322324  14610.341120       569.717148  14610.608202   
2            2  22579.583009  14436.420332       593.109370  14437.481875   
3            3  15903.338686  12059.262087       107.988834  12092.268290   
4            4  16430.003447  12287.997033        85.796567  12317.167770   
5            5  17011.994043  12494.531096        35.854266  12518.579491   
6            6  16512.453827  12305.941810        85.021631  12334.606325   
7            7  16331.642072  12230.984183        83.572211  12260.993439   
8            8  15504.916698  11928.895264       131.872749  11967.928166   
9            9  20960.495546  13937.469129       336.577891  13941.756241   
10          10  15676.198977  12121.621738        87.652851  12155.101923   
11          11  16463.178253  12455.369414        68.227393  12480.612540   

# bullshit

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')